In [11]:
import numpy as np
import pandas as pd
import teneva

In [12]:
data_path = "data/"

In [13]:
vols = np.load(data_path + "sample_lognormal_vol.npy")
vols.shape # S, T, V_atm, Beta, Rho, Volvol, Displacement, K

(3, 3, 3, 3, 3, 3, 5, 3)

In [42]:
names = ["S", "T", "V_atm", "Beta", "Rho", "Volvol", "Displacement", "K"]
multiindex = pd.MultiIndex.from_product([range(i) for i in vols.shape],
                                        names=names
                                       )
full_df = pd.DataFrame(vols.reshape((-1,1)), index=multiindex, columns=["Lognormal_vol"])
full_df.head()

Lognormal_vol
S T V_atm Beta Rho Volvol Displacement K               
0 0 0     0    0   0      0            0      20.000000
                                       1       3.737664
                                       2       2.384315
                          1            0      14.995626
                                       1       5.283900

In [52]:
# verification
vols[0,0,1,0,0,0,0,2], full_df.loc[0,0,1,0,0,0,0,2].values.item()

(8.096674598403172, 8.096674598403172)

In [73]:
missing_df = full_df.sample(frac=0.3, replace=False, random_state=1)
missing_df.shape

(3280, 1)

In [72]:
train_df = full_df.drop(missing_df.index)
train_df.shape

(7655, 1)

In [74]:
missing_df.shape[0] + train_df.shape[0] == full_df.shape[0]

True

In [76]:
Y_train = train_df.values
I_train = train_df.reset_index()[names].values
Y_train.shape, I_train.shape

((7655, 1), (7655, 8))

In [77]:
r = 4
Y_0 = teneva.rand(vols.shape, r)
nswp=5

Y_new = teneva.als(I_train, Y_train, Y_0, nswp)
get = teneva.getter(Y_new)
Y_train_pred = np.array([get(i) for i in I_train])
train_error = np.linalg.norm(Y_train_pred - Y_train)
train_error /= np.linalg.norm(Y_train)
print(f"train error {train_error:.4f}")

train error 116.1695


In [78]:
Y_test = missing_df.values
I_test = missing_df.reset_index()[names].values
Y_test.shape, I_test.shape 

((3280, 1), (3280, 8))

In [ ]:
Y_test_pred = np.array([get(i) for i in I_test])
train_error = np.linalg.norm(Y_train_pred - Y_train)
train_error /= np.linalg.norm(Y_train)
print(f"train error {train_error:.4f}")